# Scraping Thơ
trên thivien.net
- lấy url của các bài thơ
- mở url lấy nội dung thơ (chữ Hán, phiên âm Hán Việt, phiên dịch)
*trang block scrap nên phải dùng header 

In [4]:
import requests
from bs4 import BeautifulSoup
import random
import csv
import re
import pandas as pd
import requests

from docx import Document
import time



Phần scrap url

In [3]:


user_agents_list = [
    'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
]

# URL của trang web cần scrap
url = "https://www.thivien.net/L%C3%BD-B%E1%BA%A1ch/author-8WYDeeki5xZo8eD-k6Sk5A"

try:
    # Tải nội dung của trang web
    print("Đang tải nội dung trang web...")
    response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
    print("Nội dung trang web đã được tải thành công.")

    # Phân tích HTML content
    print("Phân tích HTML content...")
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất các URL từ trang web chứa từ "poem" nhưng không lấy từ "searchpoem"
    print("Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...")
    urls = [a["href"] for a in soup.find_all("a", href=True) if "poem" in a["href"] and "searchpoem" not in a["href"] and "addpoem" not in a["href"] and "m-poems" not in a["href"]]


    # Lưu các URL vào tệp CSV
    file_path = "D:/LyBach-url-poem.csv"
    with open(file_path, "w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["URL"])

        for url in urls:
            writer.writerow([url])

    print(f"Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào {file_path}")

except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Đang tải nội dung trang web...
Nội dung trang web đã được tải thành công.
Phân tích HTML content...
Trích xuất các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem'...
Các URL chứa từ 'poem' nhưng không lấy từ 'searchpoem' đã được lưu vào D:/LyBach-url-poem.csv


In [5]:
# Đọc file csv chứa các URL
df = pd.read_csv("D:/LyBach-url-poem.csv")
# Khởi tạo danh sách để lưu trữ dữ liệu
data = []
# Hàm để thêm phần đầu của URL
def add_prefix(url):
    return "https://www.thivien.net" + url

# Thêm phần đầu vào từng URL
df['Full_URL'] = df['URL'].apply(add_prefix)

# Hàm để trích xuất dữ liệu từ URL và lưu vào file .docx
def extract_poem_data(url):
    user_agents_list = [
        'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36'
    ]
    HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
    try:
        # Delay 15 giây trước khi tải nội dung trang web
        random_sleep_time = random.randint(8, 15)
        time.sleep(random_sleep_time)
       
        
        # Tải nội dung của trang web
        print("Đang tải nội dung trang web:", url)
        #response = requests.get(url, headers={'User-Agent': random.choice(user_agents_list)})
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt
   
        # Tạo biến đếm cho số lần đã mở link
        link_count = 0
        
        # Phân tích HTML content
        soup = BeautifulSoup(response.content, "html.parser")

        # Trích xuất tiêu đề
        title = soup.title.text.strip()
        print("Tiêu đề:", title)

        # Trích xuất phần văn bản chữ Hán và chữ Việt
        han_viet_text = ""
        for p_tag in soup.find_all('p', class_='HanChinese transcriptable'):
            han_viet_text += p_tag.text.strip() + "\n"

        # Tìm thẻ 
        
            
        translations = []
        h4_elements = soup.find_all("h4", string=True)
        for h4_element in h4_elements:
            if h4_element.text.strip() != "Đăng nhập" and h4_element.text.strip() != "Dịch nghĩa":
                #print("Tiêu đề:", h4_element.text.strip())
                p_tag = h4_element.find_next_sibling("p")
                
                if p_tag:
                    #print("Nội dung p:", p_tag.text.strip()) 
                    description_content =  p_tag.text.strip()
              

            if h4_element.text.strip() == "Dịch nghĩa":
                
                p_tag = h4_element.find_next_sibling("p")
                if p_tag:
                    for line in p_tag.stripped_strings:
                        content_text = line
                        translations.append(content_text)
       
        # Tạo dữ liệu mới
        poem_data = {
            'Tiêu đề': title,
            'Chữ Hán': han_viet_text,
            'Phiên âm': description_content,
            'Phiên dịch': '\n'.join(translations)
        }
        
        # Thêm dữ liệu vào danh sách
        data.append(poem_data)
        df_poems = pd.DataFrame(data)
        df_poems.to_csv("D:/dophu_poem.csv", index=False, encoding='utf-8-sig')
        print("đÃ CẬP NHẬT FILE")
         # Tăng biến đếm sau mỗi lần mở link
        link_count += 1
        # Kiểm tra nếu biến đếm chia hết cho 5 thì nghỉ ngơi 10 giây
        if link_count % 5 == 0:
            print(f"Đang chờ ngẫu nhiên từ {random_sleep_time} giây...")
           
            random_sleep_time = random.randint(15, 25)
            
           
            time.sleep(random_sleep_time)
        
    except requests.exceptions.RequestException as e:
        print("Lỗi khi tải URL:", e)
    except Exception as e:
        print("Đã xảy ra lỗi:", e)

# Gọi hàm để trích xuất dữ liệu từ từng URL và lưu vào file .docx
for url in df['Full_URL']:
    extract_poem_data(url)
# Tạo DataFrame từ danh sách dữ liệu
df_poems = pd.DataFrame(data)

# Lưu DataFrame vào tệp CSV
df_poems.to_csv("D:/lybach_poem.csv", index=False, encoding='utf-8-sig')
print("Đã lưu dữ liệu vào file D:/Lybach_poem.csv")


Đang tải nội dung trang web: https://www.thivien.net/L%C3%BD-B%E1%BA%A1ch/T%C4%A9nh-d%E1%BA%A1-t%E1%BB%A9/poem-LcnJKYyCSQXVWOfCdaW6uA
Tiêu đề: Bài thơ: Tĩnh dạ tứ - 靜夜思 (Lý Bạch - 李白)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/L%C3%BD-B%E1%BA%A1ch/Ho%C3%A0ng-H%E1%BA%A1c-l%C3%A2u-t%E1%BB%91ng-M%E1%BA%A1nh-H%E1%BA%A1o-Nhi%C3%AAn-chi-Qu%E1%BA%A3ng-L%C4%83ng/poem-aMN7s3bVCOu89mMWzbg_Kg
Tiêu đề: Bài thơ: Hoàng Hạc lâu tống Mạnh Hạo Nhiên chi Quảng Lăng - 黃鶴樓送孟浩然之廣陵 (Lý Bạch - 李白)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/L%C3%BD-B%E1%BA%A1ch/Th%C6%B0%C6%A1ng-ti%E1%BA%BFn-t%E1%BB%ADu/poem-2SOZi2_E9ZMjy3mQn1w89A
Tiêu đề: Bài thơ: Thương tiến tửu - 將進酒 (Lý Bạch - 李白)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web: https://www.thivien.net/L%C3%BD-B%E1%BA%A1ch/V%E1%BB%8Dng-L%C6%B0-s%C6%A1n-b%E1%BB%99c-b%E1%BB%91/poem-4kRXqlIz-vUI7PD0-s3-Kw
Tiêu đề: Bài thơ: Vọng Lư sơn bộc bố - 望廬山瀑布 (Lý Bạch - 李白)
đÃ CẬP NHẬT FILE
Đang tải nội dung trang web:

# Phần Test khi chỉ lấy 1 bài thơ

In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Khởi tạo DataFrame để lưu trữ dữ liệu
data = {'Tiêu đề': [], 'Nội dung p': []}
df = pd.DataFrame(data)
HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}

# URL của trang web cần lấy nội dung
url = "https://www.thivien.net/L%C3%BD-Thanh-Chi%E1%BA%BFu/Ho%C3%A1n-kh%C3%AA-sa-k%E1%BB%B3-1/poem-4I9yVriyo-i1xkAkTL_TdQ"
try:
    # Tải nội dung của trang web
    response = requests.get(url, headers=HEADERS)
    response.raise_for_status()  # Ném ngoại lệ cho các mã trạng thái không tốt

    # Phân tích HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Trích xuất tiêu đề
    title = soup.title.text.strip()
    print("Tiêu đề:", title)

    # Tìm thẻ h4 và in các đoạn văn bản <p> ngay sau mỗi thẻ h4
    h4_elements = soup.find_all("h4", string=True)
    for h4_element in h4_elements:
        if h4_element.text.strip() != "Đăng nhập":
            print("Tiêu đề:", h4_element.text.strip())
            p_tag = h4_element.find_next_sibling("p")
            if p_tag:
                print("Phiên âm:", p_tag.text.strip())
        if h4_element.text.strip() == "Dịch nghĩa":
            
            p_tag = h4_element.find_next_sibling("p")
            if p_tag:
                for line in p_tag.stripped_strings:
                    print(line)
except requests.exceptions.RequestException as e:
    print("Lỗi khi tải URL:", e)
except Exception as e:
    print("Đã xảy ra lỗi:", e)


Tiêu đề: Bài thơ: Hoán khê sa kỳ 1 - 浣溪沙其一 (Lý Thanh Chiếu - 李清照)
Tiêu đề: Hoán khê sa kỳ 1
Phiên âm: Tú mạc phù dung nhất tiếu khai,Tà ổi bảo áp sấn hương tai,Nhãn ba tài động bị nhân sai.Nhất diện phong tình thâm hữu vận,Bán tiên kiều hận ký u hoài,Nguyệt di hoa ảnh ước trùng lai.
